# Load data and packages

In [1]:
# !pip install pyarrow

In [2]:
import pandas as pd
import numpy as np

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

# Development

## Q1 - Read the data for January. How many columns are there?

In [ ]:
df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [7]:
df_january.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [8]:
df_january.shape

(3066766, 19)

Response: The january data has 19 columns

## Q2 - What's the standard deviation of the trips duration in January?

In [9]:
df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
df_january['duration'] = df_january['duration'].apply(lambda x: round(x.total_seconds()/60,4))


In [10]:
df_january['duration'].std()

42.59435126046297

Response = The standard deviation is aprox 42.59 minutes

## Q3 - What fraction of the records left after you dropped the outliers?

In [11]:
df_january_2 = df_january[(df_january["duration"]>=1) & ((df_january["duration"]<=60))]

In [12]:
print(f'{round((df_january_2.shape[0]/df_january.shape[0])*100,2)}%')

98.12%


Response: The 98% of the original observations left after dropout the outliers

## Q4 - One hot encoding - What's the dimensionality of this matrix (number of columns)?

In [13]:
print(f" The number of different pickup locations are: {df_january_2['PULocationID'].nunique()}")
print(f" The number of different dropoff locations are: {df_january_2['DOLocationID'].nunique()}")

 The number of different pickup locations are: 255
 The number of different dropoff locations are: 260


In [14]:
df_january_2['PULocationID']=df_january_2['PULocationID'].astype(str)
df_january_2['DOLocationID']=df_january_2['DOLocationID'].astype(str)

C:\Users\jhorea\AppData\Local\Temp\ipykernel_26932\1859058989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_january_2['PULocationID']=df_january_2['PULocationID'].astype(str)
C:\Users\jhorea\AppData\Local\Temp\ipykernel_26932\1859058989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_january_2['DOLocationID']=df_january_2['DOLocationID'].astype(str)


In [15]:
X_train=df_january_2[['PULocationID','DOLocationID']]

In [16]:
X_train.isna().sum()

PULocationID    0
DOLocationID    0
dtype: int64

In [17]:
enconder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
enconder.fit(X_train)
column_names=enconder.get_feature_names_out()

In [18]:
X_train = pd.DataFrame(enconder.transform(X_train),columns=column_names)

In [19]:
X_train.sample(2)

,PULocationID_1,PULocationID_10,PULocationID_100,PULocationID_101,PULocationID_102,PULocationID_106,PULocationID_107,PULocationID_108,PULocationID_109,PULocationID_11,...,DOLocationID_90,DOLocationID_91,DOLocationID_92,DOLocationID_93,DOLocationID_94,DOLocationID_95,DOLocationID_96,DOLocationID_97,DOLocationID_98,DOLocationID_99
1932096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1724913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X_train.shape

(3009173, 515)

Response: The new dataframe has 515 columns

## Q5 - Training a model - What's the RMSE on train?

In [21]:
target='duration'
y_train=df_january_2[target].values

In [22]:
del df_january
del df_january_2
# del enconder

In [23]:
model=LinearRegression()

In [24]:
model.fit(X_train,y_train)

LinearRegression()

In [25]:
mean_squared_error(y_train,model.predict(X_train))**(1/2)

7.649263610098949

Response: The Root Mean Squared Error for the base model in the training dataset is aprox 7.65

## Q6 - Evaluating the model - What's the RMSE on validation?

In [26]:
del X_train
del y_train

In [27]:
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [28]:
df_february.reset_index(drop=True,inplace=True)
df_february.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2006398,2,2023-02-20 20:09:00,2023-02-20 20:13:02,2.0,0.96,1.0,N,236,237,1,6.5,1.0,0.5,2.00,0.0,1.0,13.50,2.5,0.0
2541581,2,2023-02-25 21:31:35,2023-02-25 21:48:21,2.0,4.16,1.0,N,107,87,1,21.2,1.0,0.5,6.55,0.0,1.0,32.75,2.5,0.0
1412067,1,2023-02-14 21:39:55,2023-02-14 21:51:48,1.0,1.90,1.0,N,162,234,1,11.4,3.5,0.5,1.60,0.0,1.0,18.00,2.5,0.0
2513056,1,2023-02-25 16:41:58,2023-02-25 16:48:19,0.0,1.20,1.0,N,43,238,1,9.3,2.5,0.5,2.25,0.0,1.0,15.55,2.5,0.0
993838,2,2023-02-10 19:03:13,2023-02-10 19:10:15,1.0,1.35,1.0,N,231,249,1,9.3,2.5,0.5,3.16,0.0,1.0,18.96,2.5,0.0


In [29]:
df_february['duration'] = df_february.tpep_dropoff_datetime - df_february.tpep_pickup_datetime
df_february['duration'] = df_february['duration'].apply(lambda x: round(x.total_seconds()/60,4))
df_february_2 = df_february[(df_february["duration"]>=1) & ((df_february["duration"]<=60))]

del df_february

df_february_2['PULocationID']=df_february_2['PULocationID'].astype(str)
df_february_2['DOLocationID']=df_february_2['DOLocationID'].astype(str)
X_val=df_february_2[['PULocationID','DOLocationID']]
y_val=df_february_2[target]

del df_february_2

In [30]:
X_val = pd.DataFrame(enconder.transform(X_val),columns=column_names)

In [31]:
X_val.sample(2)

,PULocationID_1,PULocationID_10,PULocationID_100,PULocationID_101,PULocationID_102,PULocationID_106,PULocationID_107,PULocationID_108,PULocationID_109,PULocationID_11,...,DOLocationID_90,DOLocationID_91,DOLocationID_92,DOLocationID_93,DOLocationID_94,DOLocationID_95,DOLocationID_96,DOLocationID_97,DOLocationID_98,DOLocationID_99
877623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X_val.sum(axis=1).sort_values()[0:10]

1731895    1.0
2810450    1.0
1588449    1.0
2496332    1.0
2317750    1.0
1731894    1.0
2111951    1.0
1545373    1.0
1903962    2.0
1903963    2.0
dtype: float64

There are some observations in february that dont have a known combination of pickup and drop-off location.  
Due to this, this observations could corrupt the prediction of the model.  
I will remove those observations (Aprox 8 observations) from input (X_val) and y_val.

In [39]:
row_sum = X_val.sum(axis=1)
id_obs = X_val[row_sum == 1].index

X_val_ = X_val[row_sum == 2]

#  To gather the y_val taking in consideration only the valid observations
mask=np.where(y_val.index.isin(id_obs),False,True)
y_val_=y_val[mask]

del y_val
del X_val
del row_sum
del id_obs
del mask


In [41]:
mean_squared_error(y_val_,model.predict(X_val_))**(1/2)

8.240261636800494

Response: The RMSE on validation dataset is aprox 8.24. So, the closest option in homework is 7.81

In [42]:
del model
del enconder
del X_val_
del y_val_

## Q6 - Second way

My 2th way to develop this point is training a linear regression model with regularization , a Lasso model.  
I figure out that the coefficients of the base LR model are too high, and this could be improductive.  

By this way, i should not to drop the observations with unknown combinations of pickup-dropoff location in te validation dataset.

In [ ]:
df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

df_january['duration'] = df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
df_january['duration'] = df_january['duration'].apply(lambda x: round(x.total_seconds()/60,4))

df_january = df_january[(df_january["duration"]>=1) & ((df_january["duration"]<=60))]

df_january['PULocationID']=df_january['PULocationID'].astype(str)
df_january['DOLocationID']=df_january['DOLocationID'].astype(str)

X_train=df_january[['PULocationID','DOLocationID']]

enconder=OneHotEncoder(sparse_output=False,handle_unknown='ignore')
enconder.fit(X_train)
column_names=enconder.get_feature_names_out()

X_train = pd.DataFrame(enconder.transform(X_train),columns=column_names)

target='duration'
y_train=df_january[target].values

del df_january

model_2=Lasso(alpha=0.01)

model_2.fit(X_train,y_train)

print(model_2.coef_.min())
print(model_2.coef_.max())

In [48]:
mean_squared_error(y_train,model_2.predict(X_train))**(1/2)

8.015527513819556

In [55]:
del X_train
del y_train

In [56]:
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_february['duration'] = df_february.tpep_dropoff_datetime - df_february.tpep_pickup_datetime
df_february['duration'] = df_february['duration'].apply(lambda x: round(x.total_seconds()/60,4))

df_february = df_february[(df_february["duration"]>=1) & ((df_february["duration"]<=60))]

df_february['PULocationID']=df_february['PULocationID'].astype(str)
df_february['DOLocationID']=df_february['DOLocationID'].astype(str)

X_val=df_february[['PULocationID','DOLocationID']]

X_val = pd.DataFrame(enconder.transform(X_val),columns=column_names)

target='duration'
y_val=df_february[target].values

del df_february

In [64]:
mean_squared_error(y_val,model_2.predict(X_val),squared=False)

8.138096225738082

response: By this way, i reach a similar result.

# Code from original notebook

In [ ]:
# train_dicts = df[categorical + numerical].to_dict(orient='records')

# dv = DictVectorizer()
# X_train = dv.fit_transform(train_dicts)

# target = 'duration'
# y_train = df[target].values

# lr = LinearRegression()
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_train)

# mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
# sns.distplot(y_pred, label='prediction')
# sns.distplot(y_train, label='actual')

# plt.legend()

In [ ]:
# def read_dataframe(filename):
#     if filename.endswith('.csv'):
#         df = pd.read_csv(filename)

#         df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
#         df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
#     elif filename.endswith('.parquet'):
#         df = pd.read_parquet(filename)

#     df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
#     df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

#     df = df[(df.duration >= 1) & (df.duration <= 60)]

#     categorical = ['PULocationID', 'DOLocationID']
#     df[categorical] = df[categorical].astype(str)
    
#     return df

In [ ]:
# df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
# df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [ ]:
# len(df_train), len(df_val)

In [ ]:
# df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
# df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
# categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

# dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient='records')
# X_train = dv.fit_transform(train_dicts)

# val_dicts = df_val[categorical + numerical].to_dict(orient='records')
# X_val = dv.transform(val_dicts)

In [ ]:
# target = 'duration'
# y_train = df_train[target].values
# y_val = df_val[target].values

In [ ]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [ ]:
# lr = Lasso(0.01)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)